<a href="https://colab.research.google.com/github/PurnamaRidzkyN/StoreSales-TimeSeriesForecasting/blob/main/Store_Sales_Time_Series_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'store-sales-time-series-forecasting:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F29781%2F2887556%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240703%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240703T050121Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da3d22ccc1a2430710cda41fbdf9bc8bfc04f96adfc1372a4abd55054dd3f90853bde49ddd836ebf1a67055f14635d539939434f4cf626e358990411967fefe5e8fcb15aa221010c12033f902e8797940fbbde6419fd9031dbe1fe456c381f980dc7a346a73c62413e3cf4bd5446d1d9c51f25b2ba36b4cc6dd529e7a6875880be09a63c5328c9677f40f458c4114d5c8bb5466e054662350fb0e3d0be435842f8f8f5629be15a608c294052209cf92626ab9e516089cc1241cd051b647bbf7c4dc21c57a7a5d3d5d42a1734b23b998e981e6cc85c151330f46be0351eafd3edf8b1b7cc0772494caa3968fd689091d72b06b86f7cbff0a7201695331614339b8'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def create_features(df, label=None):
    """
    Creates time series features from datetime index.
    """
    df = df.copy()
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['weekday'] = df['date'].dt.day_name()
    df['weekday'] = df['weekday'].astype(cat_type)
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    df['date_offset'] = (df.date.dt.month*100 + df.date.dt.day - 320)%1300

    df['season'] = pd.cut(df['date_offset'], [0, 300, 602, 900, 1300],
                          labels=['Spring', 'Summer', 'Fall', 'Winter']
                   )
    X = df[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear','weekday',
           'season']]
    if label:
        y = df[label]
        return X, y
    return X


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
color_pal= sns.color_palette()

In [ ]:
file_holiday = '/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv'
file_oil = '/kaggle/input/store-sales-time-series-forecasting/oil.csv'
file_stores = '/kaggle/input/store-sales-time-series-forecasting/stores.csv'
file_train = '/kaggle/input/store-sales-time-series-forecasting/train.csv'
file_transactions = '/kaggle/input/store-sales-time-series-forecasting/transactions.csv'
file_test='/kaggle/input/store-sales-time-series-forecasting/test.csv'

In [ ]:

df_holiday = pd.read_csv(file_holiday)
df_stores = pd.read_csv(file_stores)
df_train = pd.read_csv(file_train)

df_test = pd.read_csv(file_test)

In [ ]:

df_holiday.info()
df_stores.info()

df_train.info()

In [ ]:
df_test

In [ ]:
df_train.tail()


In [ ]:
df_holiday.head()

In [ ]:
df_holiday= df_holiday.loc[df_holiday['transferred'] == False]

In [ ]:
df_train['sales'].unique()


In [ ]:
df_stores['city'].unique()

In [ ]:
df_train['date'] = pd.to_datetime(df_train['date'])
df_test['date'] = pd.to_datetime(df_test['date'])

In [ ]:
df_train

In [ ]:
df_train["family"].unique()

In [ ]:
df_train

In [ ]:
df_holiday

In [ ]:


# Menggabungkan df_train_concat dengan df_holiday_filtered berdasarkan kolom 'date'
df_merged = pd.merge(df_train, df_stores, on='store_nbr', how='left')
df_merged_test = pd.merge(df_test, df_stores, on='store_nbr', how='left')


In [ ]:
df_merged['type'].unique()

In [ ]:
df_holiday['date'] = pd.to_datetime(df_holiday['date'])

In [ ]:
kolom_duplikat = 'date'


df_holiday = df_holiday[df_holiday['transferred'] == False]
df_holiday = df_holiday.drop_duplicates(subset=kolom_duplikat, keep='first')


# Tampilkan hasil
print(df_holiday)

In [ ]:

start_date = '2013-01-01'
end_date = '2017-08-15'
df_holiday_filtered = df_holiday[(df_holiday['date'] >= start_date) & (df_holiday['date'] <= end_date)]


df_merge= pd.merge(df_merged, df_holiday_filtered, on='date' ,how='left')

# test
start_date = '2017-08-15'
df_holiday = df_holiday[(df_holiday['date'] > start_date)]
df_merge_test= pd.merge(df_merged_test,df_holiday,on='date',how='left')

In [ ]:
df_merge

In [ ]:
df_merge['transferred'].unique()

In [ ]:
df_merge.info()

In [ ]:
total_missing_count = df_merge.isnull().sum()
total_missing_count

In [ ]:
df_merge = df_merge.fillna(0)
df_merge_test = df_merge_test.fillna(0)


In [ ]:
df_merge.set_index('id', inplace=True)
df_merge_test.set_index('id', inplace=True)

In [ ]:
df_merge['locale_name'].unique()

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class FeatureEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, type_column='type_x', locale_column='locale'):
        self.type_column = type_column
        self.locale_column = locale_column
        self.type_mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}
        self.locale_mapping = {'National': 1, 'Regional': 2, 'Local': 3, 0: 0}

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()  # Make a copy of X to avoid changing the original data

        # Map the type_x and locale columns
        X[self.type_column] = X[self.type_column].map(self.type_mapping)
        X[self.locale_column] = X[self.locale_column].map(self.locale_mapping)

        # Handle any remaining NaNs by filling them with a default value if needed
        X[self.type_column].fillna(0,inplace= True)
        X[self.locale_column].fillna(0, inplace = True)

        return X



In [ ]:

class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.drop(columns=self.columns)

In [ ]:
df_merge

In [ ]:
df_merge['city'].unique()

In [ ]:
columns_to_drop = ['transferred', 'description', 'family']

In [ ]:
from sklearn.pipeline import Pipeline

# Membuat pipeline
pipeline = Pipeline([
    ('type_to_int', FeatureEncoder()),
    ('Dropcloumns', DropColumns(columns=columns_to_drop))
    # Anda bisa menambahkan transformer atau estimator lainnya di sini
])

# Melakukan fit dan transformasi menggunakan pipeline
df_merge = pipeline.fit_transform(df_merge)
df_merge_test=pipeline.fit_transform(df_merge_test)


In [ ]:
df_merge['locale'].unique()

In [ ]:
df_objects = df_merge.select_dtypes(include=['object'])
df_objects_test = df_merge.select_dtypes(include=['object'])

In [ ]:
from sklearn.feature_extraction import FeatureHasher

hasher = FeatureHasher(n_features=3, input_type='string')
hashed_features = hasher.fit_transform(df_objects.values.astype(str))
hashed_df = pd.DataFrame(hashed_features.toarray(), columns=['Feature_1','Feature_2','Feature_3'])
print(hashed_df)

# test
hasher = FeatureHasher(n_features=3, input_type='string')
hashed_features = hasher.fit_transform(df_objects_test.values.astype(str))
hashed_df_test = pd.DataFrame(hashed_features.toarray(), columns=['Feature_1','Feature_2','Feature_3'])


In [ ]:
df_train = pd.merge(df_merge, hashed_df, left_index=True, right_index=True)


In [ ]:
numeric_columns = df_train.select_dtypes(include=['float64','int64'])

correlation_matrix = numeric_columns.corr()

plt.figure(figsize=(12,8))
sns.heatmap(correlation_matrix, cmap="YlGnBu", annot=True, linewidths=1.0)
plt.title("Correlation between Numeric columns in the Dataset")
plt.show()

In [ ]:
df_train =df_train.drop(df_objects.columns,axis=1)

In [ ]:
df_train

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


# Mengubah kolom tanggal ke format datetime
df_train['date'] = pd.to_datetime(df_train['date'])
df_train = df_train.set_index(df_train['date'])
df_train = df_train.drop(['date'],axis=1)


In [ ]:
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    #start_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    #end_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    #print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [ ]:
df_train =reduce_mem_usage(df_train)

In [ ]:
df_train

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Lag features
df_train['lag_1'] = df_train['sales'].shift(1)
df_train['lag_2'] = df_train['sales'].shift(2)
df_train.dropna(inplace=True)

# Train-test split
X = df_train.drop(['sales'],axis=1)
y = df_train['sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Fit XGBoost model
model = xgb.XGBRegressor()



In [ ]:
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)



In [ ]:
# Evaluate the model
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Calculate MSE and MAE
mse = mean_squared_error(y_test, y_pred )
mae = mean_absolute_error(y_test, y_pred )

print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")

In [ ]:
hashed_df_test.reset_index(drop=True, inplace=True)
hashed_df_test.index = hashed_df_test.index + 3000888  # Mulai dari 3000888

# Merge dengan df_merge_test menggunakan indeks
df_test = pd.merge(df_merge_test, hashed_df_test, left_index=True, right_index=True)

In [ ]:
df_merge_test

In [ ]:
df_test =df_test.drop(df_objects_test.columns,axis=1)

In [ ]:
df_test

In [ ]:
df_test= df_test.set_index('date')


In [ ]:
# Ambil nilai terakhir dari lag features di data train
last_lag_1_train = df_train['lag_1'].iloc[-1]
last_lag_2_train = df_train['lag_2'].iloc[-1]

# Buat lag features di data test dengan menggunakan nilai terakhir dari data train
df_test['lag_1'] = last_lag_1_train
df_test['lag_2'] = last_lag_2_train

In [ ]:
df_test

In [ ]:
predicts = model.predict(df_test)

In [ ]:
df_test.reset_index(drop=True, inplace=True)
df_test.index = df_test.index + 3000888 # Buat DataFrame baru untuk hasil prediksi
predictions_df = pd.DataFrame({
    'Id': df_test.index,  # Ambil PassengerId dari data uji
    'sales': predicts  # Masukkan hasil prediksi
})

# Simpan DataFrame ke dalam file CSV
predictions_df.to_csv('/kaggle/working/predictions.csv', index=False)